# Teams

In this section you'll learn how to create a _multi-agent team_ (or simply team) using AutoGen. A team is a group of agents that work together to achieve a common goal.

We'll first show you how to create and run a team. We'll then explain how to observe the team's behavior, which is crucial for debugging and understanding the team's performance, and common operations to control the team's behavior.

```{note}
When should you use a team?
Teams are for complex tasks that require collaboration and diverse expertise.
However, they also demand more scaffolding to steer compared to single agents.
While AutoGen simplifies the process of working with teams, start with
a single agent for simpler tasks, and transition to a multi-agent team when a single agent proves inadequate.
Ensure that you have optimized your single agent with the appropriate tools
and instructions before moving to a team-based approach.
```

## Creating a Team

{py:class}`~autogen_agentchat.teams.RoundRobinGroupChat` is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

We will begin by creating a team with two {py:class}`~autogen_agentchat.agents.AssistantAgent` and a {py:class}`~autogen_agentchat.conditions.TextMentionTermination` condition that stops the team when a specific word is detected in the agent's response.

The two-agent team implements the _reflection_ pattern, a multi-agent design pattern where a critic agent evaluates the responses of a primary agent. Learn more about the reflection pattern using the [Core API](../../core-user-guide/design-patterns/reflection.ipynb).

In [1]:
%%capture
# capture magic suppresses installation output
!poetry add autogen_core autogen_ext autogen_agentchat tiktoken openai

In [2]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

## Running a Team

Let's call the {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` method
to start the team with a task.

In [3]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Write a short poem about the fall season.")
print(result)

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=103), content="Leaves of amber gently fall,  \nWhispering winds begin to call.  \nCooler nights and shorter days,  \nIn autumn’s glow, the world decays.  \n\nPumpkin patches, fields of gold,  \nStories by the fire told.  \nSweaters wrapped and cider's cheer,  \nThe harvest time is finally here.  \n\nNature dons her warmest hue,  \nPainting skies a deeper blue.  \nThough the chill begins to grow,  \nIn fall's embrace, we find a glow.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=148, completion_tokens=203), content='This poem provides a beautiful depiction of the fall season, capturing both its visual and sensory elements. Here are a few points to consider for enhancement:\n\n1. **Imagery Enhancement**: The poe

In [5]:
print(result.messages[-2].content)

Thank you for the thoughtful feedback! Here’s a revised version incorporating your suggestions:

Crimson leaves in swirling dance,  
Earth in gold, a fleeting glance.  
Crisp air carries whispers low,  
In autumn's arms, memories flow.  

Candles flicker, warm and bright,  
Spices blend in sweet delight.  
Apple pie and laughter's sway,  
Harvest dreams in twilight play.  

Footsteps crunch on paths of old,  
Stories of yore in twilight told.  
Underneath the starry dome,  
In fall's embrace, we feel at home.  

Season's end, yet hearts ignite,  
Awake the soul in autumn’s light.  
Though the chill may hint of frost,  
In every leaf, we find what’s lost.


The team runs the agents until the termination condition was met.
In this case, the team ran agents following a round-robin order until the the
termination condition was met when the word "TERMINATE" was detected in the
agent's response.
When the team stops, it returns a {py:class}`~autogen_agentchat.base.TaskResult` object with all the messages produced by the agents in the team.

## Observing a Team

Similar to the agent's {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream` method, you can stream the team's messages while it is running by calling the {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream` method. This method returns a generator that yields messages produced by the agents in the team as they are generated, with the final item being the {py:class}`~autogen_agentchat.base.TaskResult` object.

In [6]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=147) content="In whispers of amber, the leaves softly sway,  \nDancing in rhythm with the crisp autumn day.  \nThe air, rich with spices of pumpkin and clove,  \nWraps the earth gently in a golden-hued robe.  \n\nThe trees stand like torches, ablaze in the light,  \nPainting the horizon with colors so bright.  \nCider's warm comfort, the chill in the air,  \nWhisper of harvests and time to prepare.  \n\nUnder skies brushed with shades of crimson and gold,  \nThe world slows to slumber as night takes its hold.  \nIn the heart of the season, we find stillness and peace,  \nEmbracing the beauty in the year's slow release.  " type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=192, completion_tokens=180) content="This poem beautifully captures the essence of fall, painting a vivid picture

As demonstrated in the example above, you can determine the reason why the team stopped by checking the {py:attr}`~autogen_agentchat.base.TaskResult.stop_reason` attribute.

The {py:meth}`~autogen_agentchat.ui.Console` method provides a convenient way to print messages to the console with proper formatting.


In [7]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves in amber hues descend,  
Whispers in the cool breeze send,  
Crisp air dances through the trees,  
A serenade of autumn’s ease.  

Harvest moons and cozy nights,  
Frosty mornings, softer lights,  
Pumpkin spice and sweater's weave,  
Magic lingers as branches leave.  

In this season, time slows down,  
Nature wears her golden crown,  
A fleeting beauty, vivid, bright,  
Before the world succumbs to white.  
---------- critic ----------
Your poem beautifully captures the essence of the fall season with vivid imagery and a soothing rhythm. Here are a few suggestions to enhance it even further:

1. **Imagery Enhancement**: Consider adding more sensory details to evoke the scents, sounds, and feelings of fall. For example, mentioning the rustling sound of leaves or the smell of bonfires could enrich the reader's experience.

2. **Structure and Flow**: While the rhyme scheme is consis

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=103), content="Leaves in amber hues descend,  \nWhispers in the cool breeze send,  \nCrisp air dances through the trees,  \nA serenade of autumn’s ease.  \n\nHarvest moons and cozy nights,  \nFrosty mornings, softer lights,  \nPumpkin spice and sweater's weave,  \nMagic lingers as branches leave.  \n\nIn this season, time slows down,  \nNature wears her golden crown,  \nA fleeting beauty, vivid, bright,  \nBefore the world succumbs to white.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=148, completion_tokens=210), content="Your poem beautifully captures the essence of the fall season with vivid imagery and a soothing rhythm. Here are a few suggestions to enhance it even further:\n\n1. **Imagery Enhancement**:

## Resetting a Team

You can reset the team by calling the {py:meth}`~autogen_agentchat.teams.BaseGroupChat.reset` method. This method will clear the team's state, including all agents.
It will call the each agent's {py:meth}`~autogen_agentchat.base.ChatAgent.on_reset` method to clear the agent's state.

In [8]:
await team.reset()  # Reset the team for the next run.

It is usually a good idea to reset the team if the next task is not related to the previous task.
However, if the next task is related to the previous task, you don't need to reset and you can instead
resume the team.

## Stopping a Team

Apart from automatic termination conditions such as {py:class}`~autogen_agentchat.conditions.TextMentionTermination`
that stops the team based on the internal state of the team, you can also stop the team
from outside by using the {py:class}`~autogen_agentchat.conditions.ExternalTermination`.

Calling {py:meth}`~autogen_agentchat.conditions.ExternalTermination.set` 
on {py:class}`~autogen_agentchat.conditions.ExternalTermination` will stop
the team when the current agent's turn is over.
Thus, the team may not stop immediately.
This allows the current agent to finish its turn and broadcast the final message to the team
before the team stops, keeping the team's state consistent.

In [9]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Crisp whispers weave through golden trees,  
Beneath a sky of boundless blue,  
The dance of leaves on the gentle breeze,  
In hues of amber, russet, and hue.  

Pumpkins rest on fields of rust,  
Harvest scents in the cooling air,  
A tapestry of autumn's trust,  
Painted with love and nature's care.  

A quiet hush embraces the land,  
As daylight gently fades away,  
In fall's embrace, hand in hand,  
We savor the fleeting, wondrous decay.  


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=114), content="Crisp whispers weave through golden trees,  \nBeneath a sky of boundless blue,  \nThe dance of leaves on the gentle breeze,  \nIn hues of amber, russet, and hue.  \n\nPumpkins rest on fields of rust,  \nHarvest scents in the cooling air,  \nA tapestry of autumn's trust,  \nPainted with love and nature's care.  \n\nA quiet hush embraces the land,  \nAs daylight gently fades away,  \nIn fall's embrace, hand in hand,  \nWe savor the fleeting, wondrous decay.  ", type='TextMessage')], stop_reason='External termination requested')

From the ouput above, you can see the team stopped because the external termination condition was met,
but the speaking agent was able to finish its turn before the team stopped.

## Resuming a Team

Teams are stateful and maintains the conversation history and context
after each run, unless you reset the team.

You can resume a team to continue from where it left off by calling the {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` or {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream` method again
without a new task.
{py:class}`~autogen_agentchat.teams.RoundRobinGroupChat` will continue from the next agent in the round-robin order.

In [10]:
await Console(team.run_stream())  # Resume the team to continue the last task.

---------- critic ----------
This poem beautifully captures the essence of fall through its vivid imagery and serene tone. The second line, “Beneath a sky of boundless blue,” pairs well with the golden ambiance, contributing to a rich visual depiction. Similarly, the detailed description of leaves and their colors creates an enchanting visual that encapsulates the season's transformation.

The second stanza evokes the sensory joys of autumn, blending earthy scents with visual elements. However, the phrase "hues of amber, russet, and hue" could be adjusted for clarity and flow, as the repetition of "hue" feels redundant. Consider diversifying the language here to maintain the poem's rhythmic elegance. 

The final stanza beautifully depicts the serene and reflective mood of fall, inviting readers to appreciate the sense of impermanence that accompanies the season. The line "A quiet hush embraces the land" sets an introspective tone, effectively concluding the poem on a resonant note.

Ov

TaskResult(messages=[TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=237), content='This poem beautifully captures the essence of fall through its vivid imagery and serene tone. The second line, “Beneath a sky of boundless blue,” pairs well with the golden ambiance, contributing to a rich visual depiction. Similarly, the detailed description of leaves and their colors creates an enchanting visual that encapsulates the season\'s transformation.\n\nThe second stanza evokes the sensory joys of autumn, blending earthy scents with visual elements. However, the phrase "hues of amber, russet, and hue" could be adjusted for clarity and flow, as the repetition of "hue" feels redundant. Consider diversifying the language here to maintain the poem\'s rhythmic elegance. \n\nThe final stanza beautifully depicts the serene and reflective mood of fall, inviting readers to appreciate the sense of impermanence that accompanies the season. The line "A quiet hu

You can see the team resumed from where it left off in the output above,
and the first message is from the next agent after the last agent that spoke
before the team stopped.

Let's resume the team again with a new task while keeping the context about the previous task.

In [11]:
# The new task is to translate the same poem to Chinese Tang-style poetry.
await Console(team.run_stream(task="将这首诗用中文唐诗风格写一遍。"))

---------- user ----------
将这首诗用中文唐诗风格写一遍。
---------- primary ----------
肃肃秋风催金树，  
碧空无尽如画图。  
落叶翩翩逐风舞，  
丹黄妆点山河殊。  

南瓜田里锈斑陈，  
丰收气息满乾坤。  
织就秋色一幅画，  
自然妙手爱意真。  

静谧送走西斜日，  
秋光暗淡别殷勤。  
同享此时秋意浓，  
惜那瞬间美缤纷。  
---------- critic ----------
这首诗在用唐诗风格重现秋季的原版诗中表现得非常优雅，并捕捉到了秋天的宁静和美丽。您的用词精准且有很好的意境。

首联"肃肃秋风催金树，碧空无尽如画图。"形象地描绘了秋风吹拂金色树木与无尽的蓝天，把季节的转变栩栩如生地展现出来。

"落叶翩翩逐风舞，丹黄妆点山河殊。"很好地展示了叶子的颜色和动态，使人很容易想象出这一美景。

颔联与颈联继续表现了秋天的丰收和自然的关怀，"南瓜田里锈斑陈，丰收气息满乾坤。"形象生动，而"织就秋色一幅画，自然妙手爱意真。"则传递了大自然的精妙和人们对其的感激。

尾联将秋天的宁静和短暂之美表达得非常到位。"静谧送走西斜日，秋光暗淡别殷勤。"让读者感受到时间的流逝和秋天的静谧；"同享此时秋意浓，惜那瞬间美缤纷。"充满对美好瞬间的珍惜。

整体来看，您的作品既保留了原诗的意境，又充满了古诗的韵味。非常出色的改编！
---------- primary ----------
感谢您的详细评价和鼓励！很高兴这首改编的唐诗风格作品能够传达出秋天的宁静与美丽，并且保留了原诗的意境。同时，我也很享受在古诗词中重新演绎秋季的过程，希望这首诗能为您带来秋日的怡然之感。感谢您的阅读和反馈！
---------- critic ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='将这首诗用中文唐诗风格写一遍。', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=575, completion_tokens=119), content='肃肃秋风催金树，  \n碧空无尽如画图。  \n落叶翩翩逐风舞，  \n丹黄妆点山河殊。  \n\n南瓜田里锈斑陈，  \n丰收气息满乾坤。  \n织就秋色一幅画，  \n自然妙手爱意真。  \n\n静谧送走西斜日，  \n秋光暗淡别殷勤。  \n同享此时秋意浓，  \n惜那瞬间美缤纷。  ', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=711, completion_tokens=344), content='这首诗在用唐诗风格重现秋季的原版诗中表现得非常优雅，并捕捉到了秋天的宁静和美丽。您的用词精准且有很好的意境。\n\n首联"肃肃秋风催金树，碧空无尽如画图。"形象地描绘了秋风吹拂金色树木与无尽的蓝天，把季节的转变栩栩如生地展现出来。\n\n"落叶翩翩逐风舞，丹黄妆点山河殊。"很好地展示了叶子的颜色和动态，使人很容易想象出这一美景。\n\n颔联与颈联继续表现了秋天的丰收和自然的关怀，"南瓜田里锈斑陈，丰收气息满乾坤。"形象生动，而"织就秋色一幅画，自然妙手爱意真。"则传递了大自然的精妙和人们对其的感激。\n\n尾联将秋天的宁静和短暂之美表达得非常到位。"静谧送走西斜日，秋光暗淡别殷勤。"让读者感受到时间的流逝和秋天的静谧；"同享此时秋意浓，惜那瞬间美缤纷。"充满对美好瞬间的珍惜。\n\n整体来看，您的作品既保留了原诗的意境，又充满了古诗的韵味。非常出色的改编！', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=1048, completion_tokens

## Aborting a Team

You can abort a call to {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` or {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream`
during execution by setting a {py:class}`~autogen_core.CancellationToken` passed to the `cancellation_token` parameter.

Different from stopping a team, aborting a team will immediately stop the team and raise a {py:class}`~asyncio.CancelledError` exception.

```{note}
The caller will get a {py:class}`~asyncio.CancelledError` exception when the team is aborted.
```

In [12]:
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

Task was cancelled.
